# **Feature Extraction** for **Extractive Summarisation**
# **For Logistic Regression and Naive Bayes**
**Features Considered for each line:**
* Number of Verbs (POS)
* Number of Stop words (nltk stop word list)
* Number of Named Entities
* Number of Pronouns (POS)
* Position in document 
* Sentence Length

** May include in Future:**
* Discourse Cues
* Sentiment
* Salience
* Uniqueness
* Has Money

## Contents

* Parsing raw data
* POS tagging
* Stop words
* Count Named Entities

In [ ]:
import os
import numpy as np
import json
import nltk

In [ ]:
nltk.download('tagsets')
nltk.download('stopwords')

In [ ]:
from paths import *

In [ ]:
TRAIN = os.listdir(DATASET_FOLDER + 'all/training/')
DEV = os.listdir(DATASET_FOLDER + 'all/validation/')
TEST = os.listdir(DATASET_FOLDER + 'all/test/')
TRAIN_FOLDER = DATASET_FOLDER + 'all/training/'
DEV_FOLDER = DATASET_FOLDER + 'all/validation/'
TEST_FOLDER = DATASET_FOLDER + 'all/test/'

In [ ]:
len(TRAIN), len(TEST), len(DEV)

In [ ]:
# folders for abstract summaries
os.mkdir(SUMMARY_FOLDER)
os.mkdir(SUMMARY_FOLDER + 'train/')
os.mkdir(SUMMARY_FOLDER + 'test')
os.mkdir(SUMMARY_FOLDER + 'dev/')

# folders for actual text
os.mkdir(TEXT_FOLDER)
os.mkdir(TEXT_FOLDER + 'train/')
os.mkdir(TEXT_FOLDER + 'test/')
os.mkdir(TEXT_FOLDER + 'dev/')

# folders for outputs
os.mkdir(OUTPUT_FOLDER)
os.mkdir(OUTPUT_FOLDER + 'train/')
os.mkdir(OUTPUT_FOLDER + 'test/')
os.mkdir(OUTPUT_FOLDER + 'dev/')

# folders for POS tags
os.mkdir(TAGS_FOLDER)
os.mkdir(TAGS_FOLDER + 'train/')
os.mkdir(TAGS_FOLDER + 'test/')
os.mkdir(TAGS_FOLDER + 'dev/')

# folders for stop counts
os.mkdir(STOPS_FOLDER)
os.mkdir(STOPS_FOLDER + 'train')
os.mkdir(STOPS_FOLDER + 'test')
os.mkdir(STOPS_FOLDER + 'dev')

## Model dataset file:

### Introduction

This dataset contains CNN and Dailymail articles used for training a summarization system. The script used to create the dataset is modified from the release of Hermann et al. 2015.

### Format:

Each file contains four parts separated by ‘\n\n’. They are
* url of the original article;
* sentences in the article and their labels (for sentence-based extractive summarization);
* extractable highlights (for word extraction-based abstractive summarization);
* named entity mapping.

### Sentence labels

There are three labels for the sentences: 1, 2 and 0. 
* ** 1 **—-sentence should extracted; 
* ** 2 **--sentence might be extracted; 
* ** 0 **—-sentence shouldn't be extracted.

### Extractable highlights

The extractable highlights are created by examining if a word (or its morphological transformation) in the highlight appears in the article or a general purpose stop-word list, which together constitute the output space (i.e., the allowed vocabulary during summary generation).

## Loading all the Data to process

In [ ]:
entities = {}
docs = []

In [ ]:
def file_parse(file):
    global entities
    # generating data
    
    contents = file.read()
    parts = contents.split('\n\n')
    lines = parts[1].split('\n')
    output = [int(line[-1]) for line in lines]
    lines = [line[:-1].rstrip() for line in lines]
    
    # storing entities
    entity_map = parts[3].split('\n')
    for i in entity_map:
        id, name = i.split(":")[:2]
        entities[id] = name
    file.close()
    return {"lines": lines, "output": output, "summary": parts[2].split('\n')}

In [ ]:
def get_processed_files(folder, file_names):
    docs = []
    summaries = []
    for file in file_names:
        try:
            docs.append(file_parse(open(folder + file)))
        except ValueError:
            pass
    return docs

In [ ]:
TRAIN = get_processed_files(TRAIN_FOLDER, TRAIN)

In [ ]:
TEST = get_processed_files(TEST_FOLDER, TEST)
DEV = get_processed_files(DEV_FOLDER, DEV)

In [ ]:
def write_files(target_folder, string, docs, filename_pattern=None):
    if not filename_pattern: filename_pattern = string
    for doc, index in zip(docs, range(len(docs))):
        with open(target_folder + filename_pattern + '.' + str(index) + '.txt', 'w+') as file:
            for line in doc[string]:
                file.write(str(line))
                file.write('\n')

In [ ]:
write_files(SUMMARY_FOLDER + 'train/', 'summary', TRAIN)
write_files(SUMMARY_FOLDER + 'test/', 'summary', TEST)
write_files(SUMMARY_FOLDER + 'dev/', 'summary', DEV)

In [ ]:
write_files(TEXT_FOLDER + 'train/', 'lines', TRAIN, 'doc')
write_files(TEXT_FOLDER + 'test/', 'lines', TEST, 'doc')
write_files(TEXT_FOLDER + 'dev/', 'lines', DEV, 'doc')

In [ ]:
write_files(OUTPUT_FOLDER + 'train/', 'output', TRAIN, 'output')
write_files(OUTPUT_FOLDER + 'test/', 'output', TEST, 'output')
write_files(OUTPUT_FOLDER + 'dev/', 'output', DEV, 'output')

## POS data

**NOTE: Since we only using primitive information like number of verbs, number of pronouns etc, we don't need accurate tags. So we simply use the most probable tag for each word. This reduces the computation time required for POS tagging**

In [ ]:
TAGS = {}
CACHED_TAGS = {}

In [ ]:
def cache(word):
    if word not in CACHED_TAGS:
        CACHED_TAGS[word] = nltk.pos_tag([word])[0][1]
    return CACHED_TAGS[word]

In [ ]:
TRAIN_TAGS = []
TEST_TAGS = []
DEV_TAGS = []

In [ ]:
def tag_data(raw, final):
    for doc in raw:
        dic = []
        for line in doc['lines']:
            l = []
            for word in line.split():
                tag = cache(word)
                l.append(tag)
            dic.append(l)
        final.append(dic)

In [ ]:
tag_data(TRAIN, TRAIN_TAGS)
tag_data(TEST, TEST_TAGS)
tag_data(DEV, DEV_TAGS)

In [ ]:
def write(target_folder, filename_pattern, docs):
    for doc, index in zip(docs, range(len(docs))):
        with open(target_folder + filename_pattern + '.' + str(index) + '.txt', 'w+') as file:
            for line in doc:
                for tag in line:
                    file.write(str(tag))
                    file.write(' ')
                file.write('\n')

In [ ]:
write(TAGS_FOLDER + 'train/', 'tags', TRAIN_TAGS)

In [ ]:
write(TAGS_FOLDER + 'test/', 'tags', TEST_TAGS)
write(TAGS_FOLDER + 'dev/', 'tags', DEV_TAGS)

In [ ]:
del TRAIN_TAGS, TEST_TAGS, DEV_TAGS, TAGS, CACHED_TAGS

## Stop words

In [ ]:
from nltk.corpus import stopwords

In [ ]:
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def count_stops(data):
    final = []
    for doc in data:
        stops = []
        for line in doc['lines']:
            count = 0
            for word in line.split():
                if word.lower() in STOPWORDS: count += 1
            stops.append(count)
        final.append(stops)
    return final

In [ ]:
TRAIN_STOPS = count_stops(TRAIN)
TEST_STOPS = count_stops(TEST)


In [ ]:
DEV_STOPS = count_stops(DEV)

In [ ]:
def write(target_folder, filename_pattern, docs):
    for doc, index in zip(docs, range(len(docs))):
        with open(target_folder + filename_pattern + '.' + str(index) + '.txt', 'w+') as file:
            for line in doc:
                file.write(str(line))
                file.write('\n')

In [ ]:
write(STOPS_FOLDER + 'train/', 'stop_counts', TRAIN_STOPS)
write(STOPS_FOLDER + 'test/', 'stop_counts', TEST_STOPS)

In [ ]:
write(STOPS_FOLDER + 'dev/', 'stop_counts', DEV_STOPS)

In [ ]:
del TRAIN_STOPS, TEST_STOPS, DEV_STOPS